In [10]:
from office365.runtime.auth.authentication_context import AuthenticationContext
from office365.sharepoint.client_context import ClientContext
from office365.sharepoint.files.file import File
from office365.runtime.auth.client_credential import ClientCredential
from office365.runtime.client_request_exception import ClientRequestException
import datetime
import pytz
import pandas as pd
import numpy as np
import csv
import glob
import pyodbc
import os 
import json
from io import BytesIO
import io
import platform
import openpyxl
from function.PyToSp import *
from tqdm import tqdm
from urllib.parse import quote_plus
import numpy as np
import pandas as pd
from sqlalchemy import create_engine, event
import pyodbc
import requests
import inspect
from validateSamSung import *
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from urllib.parse import quote_plus
import msal
from itertools import chain
from send_email import *

<b> Connect to Azure

In [11]:
import json
f = open ('database_information.json', "r")
qq = json.loads(f.read())
f.close()
ini_cnt_str ='Driver={driver_str};Server=tcp:hkazdevsqld3vnreserch.database.windows.net,1433;database={database};Uid={username};Pwd={password};Encrypt=yes;Authentication=ActiveDirectoryPassword;Connection Timeout=30;'.format(**qq)
quoted = quote_plus(ini_cnt_str)
cnt_str = 'mssql+pyodbc:///?odbc_connect={}'.format(quoted)
engine = create_engine(cnt_str)

<b> Connect To SharePoint

In [12]:
header_BIHub = 'share_point_BIHub'
config_BIHub = read_config_json(config_path, header_BIHub)
BIHub = SharePoint(config_BIHub)
BIHub.check_connect()

Web site title: S22M Research & BI Hub _ Successful Connection!


<b> Lấy đường dẫn chứa File Gap Map

In [13]:
#Tất cả các tỉnh
relative_url = "/sites/BIHub/Shared Documents/Advisory Data/Project"
sp_object = relative_url.split('/')[2].replace('-','')
list_folder = eval(sp_object).get_content_url(relative_url,return_list_folder=True)

Folder name: /sites/BIHub/Shared Documents/Advisory Data/Project/Crawling Data
/sites/BIHub/Shared Documents/Advisory Data/Project/SAMSUNG
/sites/BIHub/Shared Documents/Advisory Data/Project/GapMap Data


In [14]:
df_summ_file = pd.DataFrame({'Name':[],'ServerRelativeUrl':[], 'TimeLastModified':[], 'ModTime':[], 'Modified_by_ID':[]})
for i in list_folder:
    df_summ_file = pd.concat([df_summ_file, eval(sp_object).get_content_url(i)])
list_file = df_summ_file['ServerRelativeUrl'].to_list()
#History file
df_query=pd.DataFrame(df_summ_file).reset_index(drop=True)
df_summ_file = df_summ_file.reset_index(drop=True)

Folder name: /sites/BIHub/Shared Documents/Advisory Data/Project/Crawling Data/News
/sites/BIHub/Shared Documents/Advisory Data/Project/Crawling Data/MST
Folder name: /sites/BIHub/Shared Documents/Advisory Data/Project/SAMSUNG/Flat file
/sites/BIHub/Shared Documents/Advisory Data/Project/SAMSUNG/Raw file
Folder name: 
Files name: /sites/BIHub/Shared Documents/Advisory Data/Project/GapMap Data/Commune.xlsx
Files name: /sites/BIHub/Shared Documents/Advisory Data/Project/GapMap Data/District.xlsx


In [15]:
url = []
for i in list_file:
    data_key = i.split('/')[-1].split('.')[0].split('_')[-1]
    if str(data_key) not in ('x'):
        url.append(i)
url   

['/sites/BIHub/Shared Documents/Advisory Data/Project/GapMap Data/Commune.xlsx',
 '/sites/BIHub/Shared Documents/Advisory Data/Project/GapMap Data/District.xlsx']

<b>Import Data

In [16]:
def remove_unformated_character_GapMap(data):
    #re.sub(pattern, repl, string, count=0, flags=0)
    list_unformated_character = ['#N/a', 'N/A', 'N/a', 'n/a', 'NA', '-'
                                 , '$-', '#DIV/0!', '0', 0, 'FALSE' 
                                 , 'Null'
                                ]
    for columns in data.columns:
        data[columns] = data[columns].replace(r'^\s*$', np.nan, regex=True)#Replacing blank values (has white space)
        data[columns] = data.loc[:,columns].fillna(value='NULL')
        data[columns] = data[columns].replace('(^\s+|\s+$)', '', regex=True)
        data[columns] = data[columns].replace(r'\s+', ' ', regex=True)#remove white space in cell + line breaks from an excel cells
        data.loc[:,columns] = data[columns].apply(lambda x: x.title() if isinstance(x, str) else x)
        data[columns] = [re.sub('\.0$', '', str(i)) if type(i) == float else i for i in data[columns]]#remove trailing .0
        #data.loc[:,columns] = data[columns].apply(lambda x: x.title() if isinstance(x, str) else x)
    for i in list_unformated_character:
        data.replace(i, np.nan, inplace=True)
    data = data.replace({np.nan: None})
    return  data

In [17]:
def remove_unicode_GapMap(data):
    data = data.fillna('NULL')
    data = data.apply(lambda x:deunicode_funtion(x) if x!=None else x)
    data = data.replace('NULL',np.nan)
    data = data.replace({np.nan: None})
    return data.replace(r'\s+', ' ', regex=True)#remove line breaks from an excel cells

In [18]:
def Generate_Additional_Columns_GapMap(data,df_summ_file,BIHub,engine,file_url):
        #Generate file Date
    project_name=file_url.split('/')[-1]
    history=df_summ_file[df_summ_file.Name==project_name]
    File_Date=history.iloc[0,3]
    data['File_Date']=File_Date
    #Generate Import Date
    today=pd.to_datetime('today').strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
    data['Import_Date']=today
    #Generate Project Name
    project_name=file_url.split('/')[-1]
    project_name=project_name.split('.')[0]
    data['File_Name']=project_name 
    return data

In [19]:
#Create empty df for checking dictionary
df_dict = pd.DataFrame(columns=['File_Name', 'Missing_Values', 'Flag'])
df_temp_flat = pd.DataFrame()
df_flat = pd.DataFrame()
#-------------------------------------------------------
'''Get data''' 

columns_that_need_unidecode = []

for file_url in tqdm(url):
    data, file_name, sector = get_data(relative_url, file_url)
    
    SheetName = file_url.split('/')[-1].split('.')[0]
    if SheetName == 'Commune':
        table_name = 'GapMap_Commnue'
        columns_that_need_unidecode=['Province','Ward','District']
    elif SheetName == 'District':
        table_name = 'GapMap_District'
        columns_that_need_unidecode = ['Province','District']
    data = remove_unformated_character_GapMap(data)
    for i in columns_that_need_unidecode:
        if i not in data.columns:
            data[i] = np.nan
            data[i] = data[i].replace({np.nan: None})
            print(colored('Column {} is added in {}','yellow').format(i, f'{file_name}'))
        else:
            pass   
        data[i] = remove_unicode(data[i])

    
    data = Generate_Additional_Columns_GapMap(data,df_summ_file,BIHub,engine,file_url)
    data.to_sql(table_name, engine, index=False, if_exists='append', schema='Fresh')

100%|██████████| 2/2 [01:32<00:00, 46.03s/it]
